In [3]:
!pip install datasets
!pip install tf-keras


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch
from transformers import BertTokenizerFast, AutoTokenizer, BertForSequenceClassification, AutoModel, Trainer, TrainingArguments
from datasets import Dataset
from tqdm.notebook import tqdm
from torch.optim import AdamW


# Importing Data

In [ ]:
import re
def clean_text(text):
    # Remove patterns at beginning of text such as "WASHINGTON (Reuters) -"
    cleaned_text = re.sub(r"^\S+(?:\s+\S+)*\s*\([^)]+\)\s*-", '', str(text))
    cleaned_text = re.sub(r"\n", "", cleaned_text)
    cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
    return cleaned_text.lower()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv('./labeled_fulldata.csv')
df['text'] = df['text'].apply(clean_text)

df

,title,text,fake_news_flag,dominant_topic,topic_label
0,N.J. Democrats divided on renewing 'Bridgegate...,a day after two former allies of new jersey g...,0,1,International Affairs
1,NO KIDDING! HERE’S WHY HILLARY SUPPORTERS Will...,wow the dems are so out of touch,1,0,U.S. Politics
2,Iraqi Army: US Hindering Advance on Mosul,the iraqi army blasted the us for troubling i...,1,1,International Affairs
3,Britain frustrated by slow pace of Brexit talk...,prime minister theresa may s top ministers ar...,0,1,International Affairs
4,Israeli Trump supporters open campaign office ...,israeli supporters of us republican president...,0,1,International Affairs
...,...,...,...,...,...
116430,THIS ONE STATEMENT IS THE KEY TO THE 2016 ELEC...,listen up people this is great lou dobbs gets ...,1,8,Social Issue
116431,Virginia court rules for Trump in travel ban d...,a us federal judge in virginia ruled on friday...,0,7,Legal and Political
116432,Compromises being reached in Iran talks,washington cnn compromises on some of the cruc...,0,1,International Affairs
116433,Republican tax plan would deal financial hit t...,the republican tax plan unveiled on thursday ...,0,4,U.S. Politics


Pre-processing data

In [ ]:
import os


model_path = '/bert_model'


# df = df[:55000]


X = df['text'].values #df['text'].values. using text will be more comprehensive but also more computaitonally intensive
y = df['fake_news_flag'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y,random_state=42)
tokenizer = None
model = None

if os.path.exists(model_path):
    # Load saved BERT model and tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('./bert_tokenizer')
    model = BertForSequenceClassification.from_pretrained('./bert_model')
else:
    # Load pre-trained BERT from hugging face
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

model.to(device)
print(list(X_train)[0])

tokens_train = tokenizer.batch_encode_plus(list(X_train),max_length = 256, padding=True, truncation = True) #max_length = 128 for text

tokens_test = tokenizer.batch_encode_plus(list(X_test), max_length = 256, padding=True, truncation = True)


train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(list(y_train))

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(list(y_test))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


not all the claims in the vice presidential debate stand up to scrutiny a look at some of them and how they compare with the factsrepublican mike pence the fact that under this past administration weve almost doubled the national debt is atrocious hillary clinton and tim kaine want more of the samethe facts as a share of the total us economy the national debt has gone up 35 percent not a doublingstill the debt has ballooned to 196 trillion this largely reflected efforts by the obama administration to stop the great recessionwould clinton similarly increase the debt not according to an analysis by the independent committee for a responsible federal budgetthe clinton plan with its tax increases would increase the gross debt  both privately and publicly held by 450 billion over 10 years mind you that is on top of an 88 trillion increase already projected by the government under current lawas for donald trump the committee says his taxcutheavy plan would increase the gross debt by 43 trill

In [10]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(X_train)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(X_test)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size)

##### Install GPU CUDA

In [7]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Python 3.11.11


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Detected platform "Kaggle". Running its setup...
Updating the package lists...
Installing nvidia-cuda-toolkit, this may take a few minutes...
Source files will be saved in "/tmp/tmp40vvmqol".


#### Training Loop

In [ ]:
optimizer = AdamW(model.parameters(), lr = 1e-5)

epochs = 14

model.train()

prev_loss = 0
curr_loss = 0
rel_error = 1

# for epoch in range(epochs):
epoch = 0
while rel_error > 0.001 or curr_loss < 0.01 or epoch > epochs:
    total_loss = 0

    progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for step, batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        batch = [r.to(device) for r in batch]
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        labels=labels)

        loss, logits = outputs[:2]
        loss.backward()

        optimizer.step()


        total_loss += loss.item()
        progress.set_postfix(loss=loss.item())
        progress.update(1)
    print(f"Epoch {epoch+1} completed. Avg loss: {total_loss / len(train_dataloader):.4f}")
    curr_loss = total_loss / len(train_dataloader)
    rel_error = abs(prev_loss - curr_loss)
    prev_loss = curr_loss
    epoch += 1

Epoch 1:   0%|          | 0/2547 [00:00<?, ?it/s]

Epoch 1 completed. Avg loss: 0.0685


Epoch 2:   0%|          | 0/2547 [00:00<?, ?it/s]

Epoch 2 completed. Avg loss: 0.0207


Epoch 3:   0%|          | 0/2547 [00:00<?, ?it/s]

KeyboardInterrupt: 

#### Testing Loop

In [ ]:
model.eval()

from sklearn.metrics import accuracy_score, classification_report

all_preds = []
all_labels = []

epoch = 0
with torch.no_grad():
    # progress = tqdm(train_dataloader, desc=f"Epoch {epoch+1}")
    for step, batch in enumerate(test_dataloader):
        #print("step, batch")


        batch = [r.to(device) for r in batch]
        input_ids, attention_mask, labels = batch
        #print("Getting outputs")

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        #print(outputs.logits)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # progress.set_postfix(loss=loss.item())
        # progress.update(1)

print("Accuracy:", accuracy_score(all_labels, all_preds))
print("Classification Report:")
print(classification_report(all_labels, all_preds))



In [ ]:
from collections import Counter
print(Counter(y_train))

In [ ]:
model.save_pretrained('./bert_model')
tokenizer.save_pretrained('./bert_tokenizer')